In [3]:
import requests     # how python goes onto the internet!  Only does static content
import re           # regex
from bs4 import BeautifulSoup # a python HTML parser (version 3)

In [2]:
# let's turn to stock prices
# http://finance.google.com/finance?q=aapl

In [3]:
symbol = 'aapl'

In [4]:
# Let's grab the raw html from the page
r = requests.get('http://finance.google.com/finance?q='+symbol) # the url of the google finance page goes in here

In [5]:
#b = BeautifulSoup(r.text, "html5lib") # create a beautifulsoup object
b = BeautifulSoup(r.text, 'html.parser') # try this line instead if you have problems

In [6]:
b.prettify() # will print the html nicely

u'<!DOCTYPE html>\n<html>\n <head>\n  <script>\n   (function(){(function(){function e(a){this.t={};this.tick=function(a,c,b){var d=void 0!=b?b:(new Date).getTime();this.t[a]=[d,c];if(void 0==b)try{window.console.timeStamp("CSI/"+a)}catch(e){}};this.tick("start",null,a)}var a,d;window.performance&&(d=(a=window.performance.timing)&&a.responseStart);var f=0<d?new e(d):new e;window.jstiming={Timer:e,load:f};if(a){var c=a.navigationStart;0<c&&d>=c&&(window.jstiming.srt=d-c)}if(a){var b=window.jstiming.load;0<c&&d>=c&&(b.tick("_wtsrt",void 0,c),b.tick("wtsrt_","_wtsrt",\nd),b.tick("tbsd_","wtsrt_"))}try{a=null,window.chrome&&window.chrome.csi&&(a=Math.floor(window.chrome.csi().pageT),b&&0<c&&(b.tick("_tbnd",void 0,window.chrome.csi().startE),b.tick("tbnd_","_tbnd",c))),null==a&&window.gtbExternal&&(a=window.gtbExternal.pageT()),null==a&&window.external&&(a=window.external.pageT,b&&0<c&&(b.tick("_tbnd",void 0,window.external.startE),b.tick("tbnd_","_tbnd",c))),a&&(window.jstiming.pt=a)}catch(

In [7]:
# find all span tags
b.findAll('span')
# b.find_all('span') for bs4 users

[<span class="gbi" id="gbn"></span>,
 <span class="gbf" id="gbf"></span>,
 <span id="gbe"></span>,
 <span class="fjfe-searchbox-button-wrapper"><span class="fjfe-searchbox-button-wrapper2"><input class="fjfe-searchbox-button" tabindex="2" type="submit" value="Get quotes"/></span></span>,
 <span class="fjfe-searchbox-button-wrapper2"><input class="fjfe-searchbox-button" tabindex="2" type="submit" value="Get quotes"/></span>,
 <span class="fjfe-recentquotes-duration">(<a href="//www.google.com/history/optout?hl=en">30 days</a>)</span>,
 <span class="fjfe-chg-toggle">chg</span>,
 <span class="fjfe-perc-toggle">%</span>,
 <span class="pr">\n<span id="ref_22144_l">109.19</span>\n</span>,
 <span id="ref_22144_l">109.19</span>,
 <span class="ch bld"><span class="chg" id="ref_22144_c">+0.11</span>\n<span class="chg" id="ref_22144_cp">(0.10%)</span>\n</span>,
 <span class="chg" id="ref_22144_c">+0.11</span>,
 <span class="chg" id="ref_22144_cp">(0.10%)</span>,
 <span class="nwp">\nReal-time:\n\

In [8]:
# the pattern we recognized from the website
re_tag = re.compile("ref_\d+_l") 

In [9]:
#this tag finds the tag with the price in it!!!!
span_tag = b.find('span', attrs={'id': re_tag}) 
# use find to return THE ONE AND ONLY span tags with an id that matches our regex
# use findAll to find all matches

In [10]:
quote = span_tag.text
quote

u'109.19'

In [11]:
print span_tag

<span id="ref_22144_l">109.19</span>


In [12]:
def get_stock_price(stock):
    r = requests.get('http://finance.google.com/finance?q='+stock)
    b = BeautifulSoup(r.text, 'html.parser')
    re_tag = re.compile("ref_\d+_l")
    span_tag = b.find('span', attrs={'id': re_tag}) 
    if span_tag:
        return span_tag.text
    else:
        return 'no stock found'

In [13]:
print get_stock_price('asdf')

no stock found


In [46]:
##### EXERCISE #####
# Make a method  get_stock_price that takes in ANY stock ticker and grabs the current price
# If the stock ticker doesn't exist, return -1








In [14]:
def get_stock_price(ticker):
    response = requests.get("http://google.com/finance?q="+ticker)
    parser = BeautifulSoup(response.text, "html.parser")
    pattern = re.compile("ref_\d+_l")
    span_tag = parser.find("span", attrs={"id":pattern})
    if span_tag:
        return span_tag.text
    else:
        return -1
    
    
    

In [15]:
for ticker in ['ge', 'spy']:
    print get_stock_price(ticker)

31.28
218.57


In [47]:
###### UFO #######

In [4]:
r = requests.get("http://www.nuforc.org/webreports/ndxe201608.html")
b = BeautifulSoup(r.text, 'html.parser')

In [5]:
print b.findAll('tr', attrs = {'valign':'TOP'})

[<tr valign="TOP">\n<td bgcolor="#FFFFCC"><font color="#000000" face="Calibri" style="FONT-SIZE:11pt"><a href="129/S129366.html">8/19/16 06:55</a></font></td>\n<td bgcolor="#FFFFCC"><font color="#000000" face="Calibri" style="FONT-SIZE:11pt">East Los Angeles</font></td>\n<td bgcolor="#FFFFCC"><font color="#000000" face="Calibri" style="FONT-SIZE:11pt">CA</font></td>\n<td bgcolor="#FFFFCC"><font color="#000000" face="Calibri" style="FONT-SIZE:11pt">Oval</font></td>\n<td bgcolor="#FFFFCC"><font color="#000000" face="Calibri" style="FONT-SIZE:11pt">5 minutes</font></td>\n<td bgcolor="#FFFFCC"><font color="#000000" face="Calibri" style="FONT-SIZE:11pt">Silver oval object.</font></td>\n<td bgcolor="#FFFFCC"><font color="#000000" face="Calibri" style="FONT-SIZE:11pt">8/19/16</font></td>\n</tr>, <tr valign="TOP">\n<td><font color="#000000" face="Calibri" style="FONT-SIZE:11pt"><a href="129/S129365.html">8/19/16 00:54</a></font></td>\n<td><font color="#000000" face="Calibri" style="FONT-SIZE:1

In [10]:
# Let's take a look at the first sighting
for tr in b.findAll('tr', attrs = {'valign':'TOP'})[:1]:
    # the findChildren method returns all children underneath it
#    print tr.text   # if there is a missing value will not know 
     for child in tr.findChildren():    ##???? Why this
         print child.text

8/19/16 06:55
8/19/16 06:55
8/19/16 06:55
East Los Angeles
East Los Angeles
CA
CA
Oval
Oval
5 minutes
5 minutes
Silver oval object.
Silver oval object.
8/19/16
8/19/16


In [19]:
# OK, it's a bit messy, Let's clean it up
# Looks like the first element is the date, the 4th is the city, 6th if state, 8th is shape (this ones blank)
# 13th is the summary

In [11]:
ufo_sightings = {
        'Date':[],
        'City':[],
        'State':[],
        'Shape':[],
        'Summary':[]
    }

for tr in b.findAll('tr', attrs = {'valign':'TOP'}):
    # the findChildren method returns all children underneath it
    ufo_sighting_info = []
    for child in tr.findChildren():
        ufo_sighting_info.append(child.text)   # is creating a list for each of them
    ufo_sightings['Date'].append(ufo_sighting_info[0])
    ufo_sightings['City'].append(ufo_sighting_info[3])
    ufo_sightings['State'].append(ufo_sighting_info[5])
    ufo_sightings['Shape'].append(ufo_sighting_info[7])
    ufo_sightings['Summary'].append(ufo_sighting_info[12])

#ufo_sightings

[u'8/19/16 06:55', u'8/19/16 06:55', u'8/19/16 06:55', u'East Los Angeles', u'East Los Angeles', u'CA', u'CA', u'Oval', u'Oval', u'5 minutes', u'5 minutes', u'Silver oval object.', u'Silver oval object.', u'8/19/16', u'8/19/16']
[u'8/19/16 00:54', u'8/19/16 00:54', u'8/19/16 00:54', u'Boca Raton', u'Boca Raton', u'FL', u'FL', u'Fireball', u'Fireball', u'2 minutes', u'2 minutes', u'Silent fireball like object slowy flew across sky, then darted off to the east, like a shooting star, leaving our atmosphere.', u'Silent fireball like object slowy flew across sky, then darted off to the east, like a shooting star, leaving our atmosphere.', u'8/19/16', u'8/19/16']
[u'8/19/16 00:54', u'8/19/16 00:54', u'8/19/16 00:54', u'Boca Raton', u'Boca Raton', u'FL', u'FL', u'Fireball', u'Fireball', u'2 minutes', u'2 minutes', u'Completely silent.  Moved across sky slowly, then suddenly pulled off at incredible (impossible) speed in an eastern direction.', u'Completely silent.  Moved across sky slowly, th

In [19]:
import pandas as pd

In [20]:
pd.DataFrame(ufo_sightings) # MAGIC

,City,Date,Shape,State,Summary
0,Providence,8/16/16 05:30,Circle,RI,"Red orb moving quickly, then stops multiple ti..."
1,Edison,8/16/16 04:10,Triangle,NJ,Lighted Triangular Crafts sighted within minu...
2,New York City (Manhattan),8/16/16 02:47,Unknown,NY,I was on the roof of my apartment building eat...
3,Cornelius,8/15/16 23:15,Circle,NC,"One round bright light, brighter than the moon..."
4,Sumter,8/15/16 23:00,,SC,As I was laying in my bed around eleven O cloc...
5,San Diego,8/15/16 21:36,Fireball,CA,Flashing fireball slowly moving over Mission V...
6,Boulder,8/15/16 21:15,Unknown,CO,Light traveling beneath the clouds without mar...
7,Gilbert,8/15/16 20:05,Circle,AZ,"Round shaped object with no blinking lights, j..."
8,Nashville,8/15/16 03:10,Light,TN,"Bright orange light in the sky, appeared to be..."
9,Marina Del Rey,8/14/16 22:45,Light,CA,Walking west on Washington blvd at Palawan way...


In [ ]:
# SOME MORE EXAMPLES

In [ ]:
# NY TIMES ARTICLES ON HOME PAGE

In [31]:
response = requests.get("http://www.nytimes.com/")
parser = BeautifulSoup(response.text, "html.parser")
for story in parser.findAll("h2", attrs={'class':'story-heading'}):
    print story.text

New Zika Cases Lead to Travel Warning for Miami Area
Pregnant Women in Florida Dread Even ‘One Mosquito Bite’
Watch: Florida Governor Discusses Zika Cases 
Headed to Miami? Here’s What You Need to Know 5:34 PM ET
Trump’s Campaign Chief Leaves Days After Shake-Up
In Coal Country, Trump Is ‘the Only Shot We Got’
Republicans Running for Congress Struggle With an ‘Albatross’
As Trump Falters, Democrats Plan to Press Fight for Supreme Court
 The Run-Up Podcast: Knowing the Trump Voter
Obamacare Plans Look Less and Less Like Employer Plans
Your Friday Evening Briefing
What to Cook This Weekend
Nine New Books That Times Editors Think You Should Read
Marveling at the Wet Rockettes of the Olympics
Apologizing, Lochte Says He Wasn’t ‘Candid’
Lochte Grabs the Spotlight, but Not as Planned 
The Latest on the Case 
Rio Today: Olympic Official in Maximum-Security Jail 
On Olympics: The Debate Over Female Testosterone Levels 
An Aging Boy Band Calls It Quits, but Japan Can’t Let Go
Ex-SEAL Forfeits $

In [28]:
# WIKIPEDIA FEATURED ARTICLE
response = requests.get("https://en.wikipedia.org/wiki/Wikipedia:Today%27s_featured_article")
parser = BeautifulSoup(response.text, "html.parser")

In [29]:
parser.find('div', attrs={'id':'mw-content-text'}).text

u'\n\n\nToday\'s featured article\n\nAt the top of the Main Page, a summarized lead section from one of Wikipedia\'s featured articles is displayed as "Today\'s featured article" (TFA). The current month\'s queue can be found here. TFAs are scheduled by the TFA coordinators, Brianboulton (Brian), Crisco 1492 (Chris) and Dank (Dan). Community discussion of suggestions takes place at the TFA requests page.\nIf you notice an error in a future TFA summary, you\'re welcome to fix it yourself, but if the mistake is in today\'s or tomorrow\'s summary, you can leave a message at WP:ERRORS to ask an administrator to fix it. The summaries are formatted as a single paragraph of around 1,150 characters (including spaces), with no reference tags or alternative names. Only the link to the specified featured article is bolded, and this must be the first link. The summary should be preceded by an appropriate image when available; fair use images are not allowed.\nThe editnotice template for Today\'s F